In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode
from sklearn.cluster import KMeans
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
#import pyLDAvis.sklearn
from sklearn.manifold import MDS
from sklearn.cluster import KMeans
import plotly.express as px



nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

/global/homes/m/masare/.conda/envs/sacct/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-30 07:34:35.885787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 07:34:36.040410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 07:34:36.041133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 07:34:36.280311: I tensorflow/core/p

In [2]:
data = pd.read_csv("/global/homes/m/masare/ticket-text-analysis/backup.csv")

In [3]:
#  selecting the 'text_data' column
text_data = data['text_data']


In [4]:
# Preprocessing function
def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)




In [5]:
# Preprocess the text data
preprocessed_data = text_data.apply(preprocess)


In [6]:
# Vectorize the text data
vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')
text_vectorized = vectorizer.fit_transform(preprocessed_data)


In [7]:
# Apply LDA for topic modeling
num_topics = 5  # Number of topics
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model.fit(text_vectorized)


LatentDirichletAllocation(n_components=5, random_state=42)

In [8]:
# Get topic distributions for each document
topic_distributions = lda_model.transform(text_vectorized)


In [9]:
# Cluster the topics using KMeans
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters, random_state=42)
topic_clusters = clustering_model.fit_predict(topic_word_matrix)


NameError: name 'topic_word_matrix' is not defined

In [ ]:
# Get the topic-word matrix
topic_word_matrix = lda_model.components_


In [ ]:
topic_word_matrix 

In [ ]:
# Perform MDS to project the topic distances to 2D space
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
topic_distances = np.zeros((num_topics, num_topics))
for i in range(num_topics):
    for j in range(num_topics):
        topic_distances[i, j] = np.linalg.norm(topic_word_matrix[i] - topic_word_matrix[j])
coords = mds.fit_transform(topic_distances)


In [ ]:
coords

In [ ]:
# Prepare DataFrame for Plotly
topics = [f'Topic {i}' for i in range(num_topics)]
df_topics = pd.DataFrame(coords, columns=['D1', 'D2'])
df_topics['Topic'] = topics
df_topics['Cluster'] = topic_clusters


In [ ]:
# Combine the 'Applications' and 'Issues' columns
data['text_data'] = data['Applications'].apply(lambda x: ' '.join(x)) + ' ' + data['Issues'].apply(lambda x: ' '.join(x))


# Preprocessing function
def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


# Preprocess the text data
data['preprocessed_text'] = data['text_data'].apply(preprocess)


# Vectorize the text data
vectorizer = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')
text_vectorized = vectorizer.fit_transform(data['preprocessed_text'])


# Apply LDA for topic modeling
num_topics = 5  # Number of topics
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_model.fit(text_vectorized)  # Fit LDA to the entire dataset


# Get the topic-word matrix
topic_word_matrix = lda_model.components_  # Use the topic-word matrix for clustering


# Cluster the topics using KMeans
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters, random_state=42)
topic_clusters = clustering_model.fit_predict(topic_word_matrix)  # Cluster the topics


# Perform MDS to project the topic distances to 2D space
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
topic_distances = np.zeros((num_topics, num_topics))
for i in range(num_topics):
    for j in range(num_topics):
        topic_distances[i, j] = np.linalg.norm(topic_word_matrix[i] - topic_word_matrix[j])  # Calculate distances between topics
coords = mds.fit_transform(topic_distances)  # Project topics into 2D space using MDS


# Prepare DataFrame for Plotly
topics = [f'Topic {i}' for i in range(num_topics)]
df_topics = pd.DataFrame(coords, columns=['D1', 'D2'])
df_topics['Topic'] = topics
df_topics['Cluster'] = topic_clusters


# Create the Intertopic Distance Map using Plotly
fig = px.scatter(df_topics, x='D1', y='D2', text='Topic', color='Cluster',
                 title='Intertopic Distance Map', 
                 labels={'D1': 'Dimension 1', 'D2': 'Dimension 2'},
                 width=800, height=600)


# Customize the layout for better visualization
fig.update_traces(textposition='top center')
fig.update_layout(legend_title_text='Cluster', legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))


fig.show()
